<a href="https://colab.research.google.com/github/MdMamunSheikh/Nurse-Care-Challenge-2020/blob/main/src/python/Nurse_care_Segmentation_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main

###**Libraries**

In [ ]:
import os, sys, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.display import clear_output
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import tensorflow as tf
import keras
import keras.layers as L
import keras.backend as K
from keras.models import Input, Model, Sequential
from tqdm.notebook import tqdm
import pytz
import warnings
warnings.simplefilter('ignore')
clear_output()

In [ ]:
tf.__version__

'2.4.1'

### Resampling

In [ ]:
segmented = pd.read_pickle('/content/drive/MyDrive/ML/Nursing care activity recognitions challenge 2020/Test/segmented.pkl', compression = 'gzip')
segmented

,user_id,s,f,x,y,z,t,period,l
0,27,2018-12-01 13:46:19.903000+00:00,2018-12-01 13:46:31.709000+00:00,"[9.002, 9.079, 9.079, 9.04, 9.04, 9.079, 9.117...","[-0.306, -0.268, -0.268, -0.22899999999999998,...","[-0.5329999999999999, -0.542, -0.542, -0.54799...","[2018-12-01 13:46:19.903000+00:00, 2018-12-01 ...",0 days 00:00:11.806000,23
1,27,2018-12-01 13:46:43.310000+00:00,2018-12-01 13:46:45.513000+00:00,"[9.079, 9.117, 9.117, 9.117, 9.079, 9.117, 9.0...","[-0.344, -0.306, -0.306, -0.306, -0.306, -0.19...","[-0.506, -0.506, -0.506, -0.506, -0.513, -0.51...","[2018-12-01 13:46:43.310000+00:00, 2018-12-01 ...",0 days 00:00:02.203000,15
2,27,2018-12-01 13:47:08.910000+00:00,2018-12-01 13:47:15.914000+00:00,"[9.155, 9.04, 9.079, 9.079, 9.04, 9.04, 9.117,...","[-0.344, -0.22899999999999998, -0.306, -0.306,...","[-0.523, -0.515, -0.52, -0.52, -0.517, -0.517,...","[2018-12-01 13:47:08.910000+00:00, 2018-12-01 ...",0 days 00:00:07.004000,20
3,27,2018-12-01 13:48:15.349000+00:00,2018-12-01 13:48:30.524000+00:00,"[9.04, 9.04, 9.117, 9.117, 9.117, 9.117, 9.04,...","[-0.344, -0.344, -0.268, -0.22899999999999998,...","[-0.499, -0.499, -0.505, -0.514, -0.514, -0.51...","[2018-12-01 13:48:15.349000+00:00, 2018-12-01 ...",0 days 00:00:15.175000,48
4,27,2018-12-01 13:48:50.120000+00:00,2018-12-01 13:48:55.520000+00:00,"[9.079, 9.117, 9.117, 9.117, 9.117, 9.079, 9.1...","[-0.268, -0.268, -0.268, -0.306, -0.306, -0.30...","[-0.509, -0.49, -0.49, -0.494, -0.494, -0.4970...","[2018-12-01 13:48:50.120000+00:00, 2018-12-01 ...",0 days 00:00:05.400000,27
...,...,...,...,...,...,...,...,...,...
15832,46,2018-12-04 12:46:24.473000+00:00,2018-12-04 12:46:37.073000+00:00,"[9.04, 9.04, 9.04, 9.079, 9.079, 9.079, 9.079,...","[-0.114, -0.114, -0.114, -0.153, -0.153, -0.15...","[-0.7390000000000001, -0.7390000000000001, -0....","[2018-12-04 12:46:24.473000+00:00, 2018-12-04 ...",0 days 00:00:12.600000,28
15833,46,2018-12-04 12:46:46.674000+00:00,2018-12-04 12:46:52.873000+00:00,"[9.002, 9.002, 9.002, 8.964, 9.002, 9.002, 9.0...","[-0.038, -0.038, -0.038, -0.038, -0.076, -0.07...","[-0.9109999999999999, -0.9109999999999999, -0....","[2018-12-04 12:46:46.674000+00:00, 2018-12-04 ...",0 days 00:00:06.199000,9
15834,46,2018-12-04 12:47:05.874000+00:00,2018-12-04 12:47:10.873000+00:00,"[9.04, 9.04, 9.04, 9.04, 9.04, 9.04, 9.04, 9.0...","[-0.153, -0.153, -0.038, -0.038, -0.114, -0.11...","[-0.895, -0.895, -0.8420000000000001, -0.84200...","[2018-12-04 12:47:05.874000+00:00, 2018-12-04 ...",0 days 00:00:04.999000,35
15835,46,2018-12-04 12:47:17.073000+00:00,2018-12-04 12:47:35.278000+00:00,"[8.964, 8.964, 9.002, 9.002, 9.002, 9.002, 9.0...","[-0.076, -0.076, -0.114, -0.114, -0.114, -0.19...","[-0.8290000000000001, -0.8290000000000001, -0....","[2018-12-04 12:47:17.073000+00:00, 2018-12-04 ...",0 days 00:00:18.205000,113


In [ ]:
s1 = segmented[12000:].reset_index(drop = True).copy()
s1

,user_id,s,f,x,y,z,t,period,l
0,46,2018-12-03 10:19:07.714000+00:00,2018-12-03 10:19:12.717000+00:00,"[-9.117, -9.117, -9.117, -9.079, -9.079, -9.15...","[0.076, 0.076, 0.076, 0.038, 0.038, 0.038, 0.0...","[-0.502, -0.502, -0.502, -0.495, -0.495, -0.49...","[2018-12-03 10:19:07.714000+00:00, 2018-12-03 ...",0 days 00:00:05.003000,36
1,46,2018-12-03 10:19:18.514000+00:00,2018-12-03 10:19:21.319000+00:00,"[-9.155, -9.155, -9.155, -9.155, -9.155, -9.15...","[0.038, 0.038, 0.038, 0.038, 0.038, 0.038, 0.0...","[-0.494, -0.494, -0.494, -0.494, -0.494, -0.49...","[2018-12-03 10:19:18.514000+00:00, 2018-12-03 ...",0 days 00:00:02.805000,42
2,46,2018-12-03 10:19:28.513000+00:00,2018-12-03 10:19:39.913000+00:00,"[-9.079, -9.079, -9.079, -9.079, -9.04, -9.04,...","[0.0, 0.0, 0.0, 0.0, 0.076, 0.076, 0.076, 0.07...","[-0.5, -0.5, -0.5, -0.5, -0.501, -0.501, -0.50...","[2018-12-03 10:19:28.513000+00:00, 2018-12-03 ...",0 days 00:00:11.400000,60
3,46,2018-12-03 10:19:45.313000+00:00,2018-12-03 10:19:52.512000+00:00,"[-9.079, -9.079, -9.079, -9.079, -9.079, -9.07...","[0.038, 0.038, 0.038, 0.038, 0.038, 0.038, 0.0...","[-0.496, -0.496, -0.496, -0.496, -0.496, -0.49...","[2018-12-03 10:19:45.313000+00:00, 2018-12-03 ...",0 days 00:00:07.199000,74
4,46,2018-12-03 10:20:00.112000+00:00,2018-12-03 10:20:04.912000+00:00,"[-9.079, -9.079, -9.079, -9.079, -9.117, -9.11...","[0.114, 0.114, 0.114, 0.114, 0.038, 0.038, 0.0...","[-0.517, -0.517, -0.517, -0.517, -0.514, -0.51...","[2018-12-03 10:20:00.112000+00:00, 2018-12-03 ...",0 days 00:00:04.800000,14
...,...,...,...,...,...,...,...,...,...
3832,46,2018-12-04 12:46:24.473000+00:00,2018-12-04 12:46:37.073000+00:00,"[9.04, 9.04, 9.04, 9.079, 9.079, 9.079, 9.079,...","[-0.114, -0.114, -0.114, -0.153, -0.153, -0.15...","[-0.7390000000000001, -0.7390000000000001, -0....","[2018-12-04 12:46:24.473000+00:00, 2018-12-04 ...",0 days 00:00:12.600000,28
3833,46,2018-12-04 12:46:46.674000+00:00,2018-12-04 12:46:52.873000+00:00,"[9.002, 9.002, 9.002, 8.964, 9.002, 9.002, 9.0...","[-0.038, -0.038, -0.038, -0.038, -0.076, -0.07...","[-0.9109999999999999, -0.9109999999999999, -0....","[2018-12-04 12:46:46.674000+00:00, 2018-12-04 ...",0 days 00:00:06.199000,9
3834,46,2018-12-04 12:47:05.874000+00:00,2018-12-04 12:47:10.873000+00:00,"[9.04, 9.04, 9.04, 9.04, 9.04, 9.04, 9.04, 9.0...","[-0.153, -0.153, -0.038, -0.038, -0.114, -0.11...","[-0.895, -0.895, -0.8420000000000001, -0.84200...","[2018-12-04 12:47:05.874000+00:00, 2018-12-04 ...",0 days 00:00:04.999000,35
3835,46,2018-12-04 12:47:17.073000+00:00,2018-12-04 12:47:35.278000+00:00,"[8.964, 8.964, 9.002, 9.002, 9.002, 9.002, 9.0...","[-0.076, -0.076, -0.114, -0.114, -0.114, -0.19...","[-0.8290000000000001, -0.8290000000000001, -0....","[2018-12-04 12:47:17.073000+00:00, 2018-12-04 ...",0 days 00:00:18.205000,113


In [ ]:
s1.to_pickle('/content/drive/MyDrive/ML/Nursing care activity recognitions challenge 2020/Test/Segmentation/s5.pkl', compression='gzip')

### Windowing

In [ ]:
def windowing(df, size = '5s', rate = '50ms', tol = None, pad = '1s'):
    combined = pd.DataFrame()
    if tol is None:
        tol = size
    for i in tqdm(range(len(df))):
        s, f = df.s[i], df.f[i]
        n = int(np.ceil(((f-s) - size)/ pad)) + 1
        merged = pd.DataFrame({"x":df.x[i], "y":df.y[i], "z":df.z[i], "t":df.t[i]})
        for j in range(n):
            try:
                a = s + j* pd.Timedelta(pad)
                b = a + pd.Timedelta(size) 
                if j == n-1 and f - s < pd.Timedelta(tol):
                    continue
                seg = merged.loc[(merged.t>=a) & (merged.t<=b)]
                first = seg.iloc[0, :]
                first.t = a
                last = seg.iloc[len(seg) - 1, :]
                last.t = b - pd.Timedelta(rate)
                seg = seg.append(first, ignore_index = True)
                seg = seg.append(last, ignore_index = True)
                seg = seg.drop_duplicates()
                seg.set_index('t', inplace = True)
                seg = seg.resample(rate).bfill(limit = 1).interpolate(method = 'linear')
                if seg.shape[0] % 100  == 1:
                    combined = combined.append({
                        'user_id':df.user_id[i],
                        's':a,
                        'f':b,
                        'x':seg.x.values[:-1],
                        'y':seg.y.values[:-1],
                        'z':seg.z.values[:-1],
                        't':seg.index.values[:-1]
                    }, ignore_index = True)
                else:
                    combined = combined.append({
                        'user_id':df.user_id[i],
                        's':a,
                        'f':b,
                        'x':seg.x.values,
                        'y':seg.y.values,
                        'z':seg.z.values,
                        't':seg.index.values
                    }, ignore_index = True)
                
            except Exception as error:
                continue
    combined['l'] = combined.t.apply(len)
    return combined

In [ ]:
s2 = pd.read_pickle('/content/drive/MyDrive/ML/Nursing care activity recognitions challenge 2020/Test/Segmentation/s2.pkl', compression= 'gzip')
s2.shape

(3000, 9)

In [ ]:
r2 = windowing(s2, size = '10s')
r2

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,f,s,t,user_id,x,y,z,l
0,2018-12-02 22:04:46.730000+00:00,2018-12-02 22:04:36.730000+00:00,"[2018-12-02T22:04:36.700000000, 2018-12-02T22:...",27.0,"[9.04, 9.04975, 9.0595, 9.06925, 9.079, 9.0692...","[-0.38299999999999995, -0.33499999999999996, -...","[-0.524, -0.5245, -0.525, -0.5255000000000001,...",200
1,2018-12-02 22:04:47.730000+00:00,2018-12-02 22:04:37.730000+00:00,"[2018-12-02T22:04:37.700000000, 2018-12-02T22:...",27.0,"[9.117, 9.117, 9.117, 9.117, 9.117, 9.10750000...","[-0.268, -0.268, -0.268, -0.268, -0.268, -0.26...","[-0.535, -0.535, -0.535, -0.535, -0.535, -0.53...",200
2,2018-12-02 22:04:48.730000+00:00,2018-12-02 22:04:38.730000+00:00,"[2018-12-02T22:04:38.700000000, 2018-12-02T22:...",27.0,"[9.079, 9.079, 9.079, 9.079, 9.079, 9.079, 9.0...","[-0.22899999999999998, -0.22899999999999998, -...","[-0.531, -0.531, -0.531, -0.531, -0.531, -0.53...",200
3,2018-12-02 22:04:49.730000+00:00,2018-12-02 22:04:39.730000+00:00,"[2018-12-02T22:04:39.700000000, 2018-12-02T22:...",27.0,"[9.079, 9.079, 9.079, 9.079, 9.079, 9.079, 9.0...","[-0.22899999999999998, -0.22899999999999998, -...","[-0.531, -0.531, -0.531, -0.531, -0.531, -0.53...",200
4,2018-12-02 22:04:50.730000+00:00,2018-12-02 22:04:40.730000+00:00,"[2018-12-02T22:04:40.700000000, 2018-12-02T22:...",27.0,"[9.079, 9.079, 9.079, 9.079, 9.079, 9.079, 9.0...","[-0.22899999999999998, -0.22899999999999998, -...","[-0.531, -0.531, -0.531, -0.531, -0.531, -0.53...",200
...,...,...,...,...,...,...,...,...
65972,2018-12-04 09:56:06.750000+00:00,2018-12-04 09:55:56.750000+00:00,"[2018-12-04T09:55:56.750000000, 2018-12-04T09:...",27.0,"[9.079, 9.079, 9.079, 9.079, 9.079, 9.079, 9.0...","[-0.153, -0.14742857142857144, -0.141857142857...","[-0.515, -0.516, -0.517, -0.518, -0.519, -0.52...",200
65973,2018-12-04 09:56:07.750000+00:00,2018-12-04 09:55:57.750000+00:00,"[2018-12-04T09:55:57.750000000, 2018-12-04T09:...",27.0,"[9.117, 9.117, 9.117, 9.117, 9.117, 9.117, 9.1...","[-0.153, -0.153, -0.153, -0.153, -0.153, -0.15...","[-0.514, -0.514, -0.514, -0.514, -0.514, -0.51...",200
65974,2018-12-04 09:56:08.750000+00:00,2018-12-04 09:55:58.750000+00:00,"[2018-12-04T09:55:58.750000000, 2018-12-04T09:...",27.0,"[9.117, 9.117, 9.117, 9.117, 9.117, 9.117, 9.1...","[-0.153, -0.153, -0.153, -0.153, -0.153, -0.15...","[-0.514, -0.514, -0.514, -0.514, -0.514, -0.51...",200
65975,2018-12-04 09:56:09.750000+00:00,2018-12-04 09:55:59.750000+00:00,"[2018-12-04T09:55:59.750000000, 2018-12-04T09:...",27.0,"[9.117, 9.1265, 9.136, 9.1455, 9.155, 9.155, 9...","[-0.153, -0.1625, -0.172, -0.1815, -0.191, -0....","[-0.514, -0.5125, -0.511, -0.5095000000000001,...",200


In [ ]:
r2.to_pickle('/content/drive/MyDrive/ML/Nursing care activity recognitions challenge 2020/Test/Resampling/r2.pkl', compression = 'gzip')


In [ ]:
test = pd.read_pickle('/content/drive/MyDrive/ML/Nursing care activity recognitions challenge 2020/Test/PKL/test.pkl', compression = 'gzip')
pred = pd.read_pickle('/content/drive/MyDrive/ML/Nursing care activity recognitions challenge 2020/Test/PKL/pred2.pkl', compression = 'gzip')

for i in tqdm(range(len(pred))):
    user_id = pred.user_id[i]
    s, f = pred.s[i], pred.f[i]
    test.pred[(test.user_id == user_id)  & (test.datetime >= s) & (test.datetime < f)] = pred.label[i]
test.to_pickle('/content/drive/MyDrive/ML/Nursing care activity recognitions challenge 2020/Test/PKL/test2.pkl', compression = 'gzip')